# Notebook Imports

In [2]:
import pandas as pd
import numpy as np

# Constants

In [3]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500


# Read and Load Features from .txt Files into NumPy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_train_data.shape[0]

258363

In [7]:
print('Number of rows in Training File', sparse_train_data.shape[0])
print('Number of rows in Test File', sparse_test_data.shape[0])

Number of rows in Training File 258363
Number of rows in Test File 117779


In [8]:
print('Number of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file 4013


In [9]:
print('Number of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Number of emails in test file 1724


# How to create an empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword Arguments:
    Sparse_matrix -- numpy array
    nr_words -- size of vocabulary or number of tokens.
    doc_idx -- position of the doument id on the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label(spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, nr_words))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index('DOC_ID', inplace=True)    
    return full_matrix
    

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 17.7 s


In [17]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,1,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
full_train_data[0][4]

5

# Training the Naive Bayes Model

## Calculating the probability of Spam

In [19]:
#prob_spam = len(full_train_data[full_train_data.CATEGORY == 1])/len(full_train_data)
#print('Probability of spam is', round(prob_spam, 2))

In [20]:
#Probability of ham
prob_ham = len(full_train_data[full_train_data.CATEGORY == 0])/len(full_train_data)
print('Probability of ham is', prob_ham)

Probability of ham is 0.689010715175679


In [21]:
# instructor solution 

prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size

print('Probability of spam is', round(prob_spam, 2))

Probability of spam is 0.31


# Total number of Words / Tokens

In [22]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [24]:
email_lengths[-5:]

DOC_ID
5789     18
5790     72
5791     46
5794     57
5795    264
dtype: int64

In [25]:
total_wc = email_lengths.sum()
total_wc

429231

## Number of Tokens in Spam & Ham Emails

In [26]:
#Instructor's solution to challenge

spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [27]:
spam_wc = spam_lengths.sum()
spam_wc

176351

In [28]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [29]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252880

In [30]:
spam_wc+nonspam_wc - total_wc

0

In [31]:
spam_lengths.describe()

count    1248.000000
mean      141.306891
std       264.879391
min         1.000000
25%        43.000000
50%        76.000000
75%       150.000000
max      4885.000000
dtype: float64

In [32]:
ham_lengths.describe()

count    2765.000000
mean       91.457505
std       246.553199
min         1.000000
25%        29.000000
50%        54.000000
75%        91.000000
max      6869.000000
dtype: float64

In [33]:
print('Average number of words in spam', round(spam_lengths.mean()))
print('Average number of words in ham', round(ham_lengths.mean()))

Average number of words in spam 141
Average number of words in ham 91


In [34]:
spam_wc / spam_lengths.shape[0]

141.30689102564102

In [35]:
nonspam_wc / ham_lengths.shape[0]

91.45750452079567

## Summing the tokens occuring in Spam

In [36]:
full_train_features.shape

(4013, 2500)

In [37]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
train_spam_tokens.shape

(1248, 2500)

### On Laplace smoothing, 

Since some of the tokens that appear in spam tokens will not be in the vocabulory they will have 0 which will not work well when we start working on probabilities. To smooth things out we add 1 to smooth out the data. 

In [44]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) # without laplace smoothing
summed_spam_tokens.shape

(2500,)

In [46]:
summed_spam_tokens.min()

0

In [47]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 #laplace smoothing
summed_spam_tokens.shape

(2500,)

In [48]:
summed_spam_tokens.min()

1

## Summing the Tokens occuring in Ham

In [50]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.shape

(2765, 2500)

In [51]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) # Without laplace smoothing
summed_ham_tokens.shape

(2500,)

In [52]:
summed_ham_tokens.min()

0

In [54]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 # With laplace smoothing
summed_ham_tokens.shape

(2500,)

In [57]:
summed_ham_tokens.min()

1

In [58]:
summed_ham_tokens.head()

0    5484
1    2590
2    2045
3     938
4    1612
dtype: int64

In [59]:
summed_ham_tokens.tail()

2495    14
2496     6
2497    29
2498     9
2499    25
dtype: int64

In [60]:
summed_spam_tokens[0]

2179

In [76]:
summed_spam_tokens.shape

(2500,)

## P(Token | Spam) - Probability that a token occurs given the Email is Spam

In [61]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012183
1    0.005228
2    0.006805
3    0.011306
4    0.006816
dtype: float64

In [75]:
prob_tokens_spam.shape

(2500,)

In [62]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a token occurs given the Email is NonSpam

In [63]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam

0       0.021474
1       0.010142
2       0.008008
3       0.003673
4       0.006312
          ...   
2495    0.000055
2496    0.000023
2497    0.000114
2498    0.000035
2499    0.000098
Length: 2500, dtype: float64

In [64]:
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs

In [65]:
full_train_features.sum(axis=0)

0       7661
1       3523
2       3260
3       2958
4       2829
        ... 
2495      28
2496      28
2497      28
2498      30
2499      29
Length: 2500, dtype: int64

In [66]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [67]:
prob_tokens_all

0       0.017848
1       0.008208
2       0.007595
3       0.006891
4       0.006591
          ...   
2495    0.000065
2496    0.000065
2497    0.000065
2498    0.000070
2499    0.000068
Length: 2500, dtype: float64

In [68]:
prob_tokens_all.sum()

0.9999999999999999

# Save The Trained Model

In [69]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Preparing Test Data

In [70]:
%%time

full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 8.02 s


In [71]:
full_test_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5783,0,2,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,0,0,0,0,0
5792,0,2,2,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5793,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,1,0,0


In [72]:
full_test_features = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
full_test_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
full_test_data.CATEGORY

DOC_ID
8       1
12      1
14      1
15      1
17      1
       ..
5783    0
5786    0
5788    0
5792    0
5793    0
Name: CATEGORY, Length: 1724, dtype: int64

In [74]:
np.savetxt(TEST_FEATURE_MATRIX, full_test_features)
np.savetxt(TEST_TARGET_FILE, full_test_data.CATEGORY)